jupyter nbconvert --to notebook --execute Watersheds.ipynb --ExecutePreprocessor.timeout=-1 --ExectuePreprocessor.kernel_name=ILWA

# Tasks

1. Climate Variable Visualization
2. LOCA Bias Correction

## Climate Variable Visualization

We have the daily dataset which contains the aggregated ppt, tmin/max, and swe variables that are used in modeling.  
The tasks is to create dynamic and interactive plots for a user to vizualize these climate variables.  
We will likely want to have basic zoom and and variables selection capability.  
Also, since these are aggregated to watersheds, we will also want to visualize the watersheds (maybe make an interactive paired map).
This is pretty open ended, so have fun with it!

## LOCA Bias Correction

We have historical climate variables that are distributed in some way.  
Out future climate scenarios are generated by the LOCA model, which is known to be biased.  
The LOCA model overlaps the real data, allowing us to identify how much bias there is, and hopefully correct for it.
The task is to visualize and qunatify the distributions of the real historical data, and then vis/quant the LOCA variables where they overlap.  
Once we know how these variables are biased, we then need to figure out a way to correct for this bias.

In [1]:
import geopandas as gpd
import rasterio
from rasterio.features import geometry_mask
import matplotlib.pyplot as plt
import numpy as np
from skimage.morphology import label
import json
from tqdm import tqdm
import os
from scipy.interpolate import griddata
import pandas as pd

In [ ]:
# df = pd.read_csv("/home/ScoutJarman/Code/ILWA/data/other/usgs_data.csv", parse_dates=['datetime'])

In [ ]:
# df.columns

In [ ]:
# tmp = df[['datetime', '309486_00062_00003']].dropna()
# plt.plot(tmp['datetime'], tmp['309486_00062_00003'])

In [ ]:
# plt.plot(df['309486_00062_00003'].dropna())

In [ ]:
# shapefile_path = "/home/ScoutJarman/Code/ILWA/data/shapefiles/id_ut8_1601.shp"

# watersheds = gpd.read_file(shapefile_path)
# watersheds

In [ ]:
# raster_path = "/home/ScoutJarman/Code/ILWA/data/rasters/UA/swe_81-23/swe19811028.tif"

# with rasterio.open(raster_path) as handle:
#     raster_arr = handle.read(1)
#     meta = handle.meta

# plt.imshow(raster_arr)
# meta


In [ ]:
# seg = np.zeros_like(raster_arr)
# for idx, geometry in enumerate(watersheds['geometry']):
#     mask = geometry_mask([geometry], out_shape=raster_arr.shape, transform=meta['transform'], invert=True)
#     seg[mask] = idx + 1

# seg = label(seg, background=-1) - 1
# plt.imshow(seg, cmap='Set1')

In [ ]:
# agg_img = np.zeros_like(raster_arr)
# means = []
# for l in np.unique(seg):
#     m = raster_arr[seg == l].mean()
#     means.append(m)
#     agg_img[seg == l] = m


# fig, axs = plt.subplots(1, 3)
# fig.set_size_inches(15, 5)

# axs[1].imshow(agg_img, cmap='hot', vmin=raster_arr.min(), vmax=raster_arr.max())
# axs[1].set_title("Averaged by Watershed")
# img = axs[0].imshow(raster_arr, cmap='hot')
# axs[0].set_title("Precip")
# plt.colorbar(img, ax=axs[0])
# axs[2].plot(means)
# axs[2].set_xlabel("Segment Label")
# axs[2].set_ylabel("Average Precip")

# Read in SWE

In [ ]:
# # Read layer names from the JSON file
# with open("/home/ScoutJarman/Code/ILWA/data/rasters/UA/ua_swe_daily_bearlake.tif.aux.json", 'r') as json_file:
#     layer_names = json.load(json_file)['time']

# input_path = "/home/ScoutJarman/Code/ILWA/data/rasters/UA/ua_swe_daily_bearlake.tif"
# output_directory = "/home/ScoutJarman/Code/ILWA/data/rasters/UA/swe_81-23/"

# if not os.path.exists(output_directory):
#     os.makedirs(output_directory)

In [ ]:
# # Open the original raster file
# with rasterio.open(input_path) as src:
#     # Loop through bands and layer names
#     for band_idx, layer_name in tqdm(zip(range(1, src.count + 1), layer_names), total=src.count):
#         # Read the band data
#         band_data = src.read(band_idx)

#         # Do nearest neighbor interpolation for lake values
#         non_nan_indices = np.where(~np.isnan(band_data))
#         points = np.column_stack((non_nan_indices[1], non_nan_indices[0]))
#         values = band_data[non_nan_indices]
#         rows, cols = band_data.shape
#         grid_x, grid_y = np.meshgrid(np.arange(cols), np.arange(rows))
#         interpolated_values = griddata(points, values, (grid_x, grid_y), method='nearest')
#         band_data[np.isnan(band_data)] = interpolated_values[np.isnan(band_data)]

#         year, month, day = layer_name.split("-")
#         # Create a new raster file for each band with the corresponding layer name
#         output_raster_path = f"{output_directory}/swe{int(year)}{int(month):02d}{int(day):02d}.tif"

#         # Write the band data to the new raster file with the same metadata
#         with rasterio.open(output_raster_path, 'w', **src.profile) as dst:
#             dst.write(band_data, 1)

# Format LOCA

In [ ]:
# Read layer names from the JSON file
for in_var, out_var in zip(["pr", "tasmax", "tasmin"], ['ppt', 'tmax', 'tmin']):
    output_directories = [f"/home/ScoutJarman/Code/ILWA/data/rasters/LOCA/{out_var}_{i}/" for i in ["r1", "r2", "r3"]]
    
    r1_tiffs = [
        f"/home/ScoutJarman/Code/ILWA/data/rasters/bear_lake/{in_var}/historical/day/bearlake_pr.ACCESS-CM2.historical.r1i1p1f1.1950-2014.LOCA_16thdeg_v20220519.tif",
        f"/home/ScoutJarman/Code/ILWA/data/rasters/bear_lake/{in_var}/future/day/bearlake_pr.ACCESS-CM2.ssp585.r1i1p1f1.2015-2044.LOCA_16thdeg_v20220519.tif",
        f"/home/ScoutJarman/Code/ILWA/data/rasters/bear_lake/{in_var}/future/day/bearlake_pr.ACCESS-CM2.ssp585.r1i1p1f1.2045-2074.LOCA_16thdeg_v20220519.tif",
        f"/home/ScoutJarman/Code/ILWA/data/rasters/bear_lake/{in_var}/future/day/bearlake_pr.ACCESS-CM2.ssp585.r1i1p1f1.2075-2100.LOCA_16thdeg_v20220519.tif"
    ]
    r2_tiffs = [
        f"/home/ScoutJarman/Code/ILWA/data/rasters/bear_lake/{in_var}/historical/day/bearlake_pr.ACCESS-CM2.historical.r2i1p1f1.1950-2014.LOCA_16thdeg_v20220519.tif",
        f"/home/ScoutJarman/Code/ILWA/data/rasters/bear_lake/{in_var}/future/day/bearlake_pr.ACCESS-CM2.ssp585.r2i1p1f1.2015-2044.LOCA_16thdeg_v20220519.tif",
        f"/home/ScoutJarman/Code/ILWA/data/rasters/bear_lake/{in_var}/future/day/bearlake_pr.ACCESS-CM2.ssp585.r2i1p1f1.2045-2074.LOCA_16thdeg_v20220519.tif",
        f"/home/ScoutJarman/Code/ILWA/data/rasters/bear_lake/{in_var}/future/day/bearlake_pr.ACCESS-CM2.ssp585.r2i1p1f1.2075-2100.LOCA_16thdeg_v20220519.tif"
    ]
    r3_tiffs = [
        f"/home/ScoutJarman/Code/ILWA/data/rasters/bear_lake/{in_var}/historical/day/bearlake_pr.ACCESS-CM2.historical.r3i1p1f1.1950-2014.LOCA_16thdeg_v20220519.tif",
        f"/home/ScoutJarman/Code/ILWA/data/rasters/bear_lake/{in_var}/future/day/bearlake_pr.ACCESS-CM2.ssp585.r3i1p1f1.2015-2044.LOCA_16thdeg_v20220519.tif",
        f"/home/ScoutJarman/Code/ILWA/data/rasters/bear_lake/{in_var}/future/day/bearlake_pr.ACCESS-CM2.ssp585.r3i1p1f1.2045-2074.LOCA_16thdeg_v20220519.tif",
        f"/home/ScoutJarman/Code/ILWA/data/rasters/bear_lake/{in_var}/future/day/bearlake_pr.ACCESS-CM2.ssp585.r3i1p1f1.2075-2100.LOCA_16thdeg_v20220519.tif"
    ]

    for i, tiffs in enumerate([r1_tiffs, r2_tiffs, r3_tiffs]):
        # Check and make outputdirectory for r level
        output_directory = output_directories[i]
        if not os.path.exists(output_directory):
            print(f"Making {output_directory}")
            os.makedirs(output_directory)
        # Loop through each of the different tiff files
        for input_path in tiffs:
            with open(input_path + ".aux.json", 'r') as json_handle:
                layer_names = json.load(json_handle)['time']
            # Open tiff and write each layer to new file
            with rasterio.open(input_path) as src:
                # Loop through bands and layer names
                for band_idx, layer_name in tqdm(zip(range(1, src.count + 1), layer_names), total=src.count):
                    # Read the band data
                    band_data = src.read(band_idx)

                    # Do nearest neighbor interpolation for lake values
                    non_nan_indices = np.where(~np.isnan(band_data))
                    points = np.column_stack((non_nan_indices[1], non_nan_indices[0]))
                    values = band_data[non_nan_indices]
                    rows, cols = band_data.shape
                    grid_x, grid_y = np.meshgrid(np.arange(cols), np.arange(rows))
                    interpolated_values = griddata(points, values, (grid_x, grid_y), method='nearest')
                    band_data[np.isnan(band_data)] = interpolated_values[np.isnan(band_data)]

                    year, month, day = layer_name.split("-")
                    # Create a new raster file for each band with the corresponding layer name
                    output_raster_path = f"{output_directory}/{out_var}{int(year)}{int(month):02d}{int(day):02d}.tif"

                    # Write the band data to the new raster file with the same metadata
                    with rasterio.open(output_raster_path, 'w', **src.profile) as dst:
                        dst.write(band_data, 1)
        

In [10]:
import os

def get_tif_files(directory, var_n):
    pt_tif_files = []
    for filename in os.listdir(directory):
        if var_n in filename and filename.endswith(".tif"):
            pt_tif_files.append(os.path.join(directory, filename))
    return pt_tif_files

for v in ["pr", "tasmax", "tasmin"]:
    all_files += get_tif_files("/data/ScoutJarman/LOCA/NCAR/met/ACCESS1-0/historical/", v)
all_files += get_tif_files("/data/ScoutJarman/LOCA/NCAR/vic/ACCESS1-0/historical/", "SWE")

for v in ["pr", "tasmax", "tasmin"]:
    all_files += get_tif_files(f"/data/ScoutJarman/LOCA/NCAR/met/ACCESS1-0/{scenario}/", v)
all_files += get_tif_files(f"/data/ScoutJarman/LOCA/NCAR/vic/ACCESS1-0/{scenario}/", "SWE")
# all_files

In [ ]:
# Read layer names from the JSON file
for in_var, out_var in zip(["pr", "tasmax", "tasmin", "SWE"], ['ppt', 'tmax', 'tmin', 'swe']):
    output_directories = [f"/data/ScoutJarman/LOCA/LOCA/{out_var}_{i}/" for i in ["rcp45", "rcp85"]]
    for directory in output_directories:
        if not os.path.exists(directory):
            print(f"Making {directory}")
            os.makedirs(directory)
    
    if in_var != "SWE":
        tiffs_45 = get_tif_files("/data/ScoutJarman/LOCA/NCAR/met/ACCESS1-0/historical/", in_var)
        tiffs_45 += get_tif_files(f"/data/ScoutJarman/LOCA/NCAR/met/ACCESS1-0/rcp45/", in_var)
        tiffs_85 = get_tif_files("/data/ScoutJarman/LOCA/NCAR/met/ACCESS1-0/historical/", in_var)
        tiffs_85 += get_tif_files(f"/data/ScoutJarman/LOCA/NCAR/met/ACCESS1-0/rcp85/", in_var)
    else:
        tiffs_45 = get_tif_files("/data/ScoutJarman/LOCA/NCAR/vic/ACCESS1-0/historical/", in_var)
        tiffs_45 += get_tif_files(f"/data/ScoutJarman/LOCA/NCAR/vic/ACCESS1-0/rcp45/", in_var)
        tiffs_85 = get_tif_files("/data/ScoutJarman/LOCA/NCAR/vic/ACCESS1-0/historical/", in_var)
        tiffs_85 += get_tif_files(f"/data/ScoutJarman/LOCA/NCAR/vic/ACCESS1-0/rcp85/", in_var)

    for i, tiffs in enumerate([tiffs_45, tiffs_85]):
        output_directory = output_directories[i]
        # Loop through each of the different tiff files
        for input_path in tiffs:
            with open(input_path + ".aux.json", 'r') as json_handle:
                layer_names = json.load(json_handle)['time']
            # Open tiff and write each layer to new file
            with rasterio.open(input_path) as src:
                # Loop through bands and layer names
                for band_idx, layer_name in tqdm(zip(range(1, src.count + 1), layer_names), total=src.count):
                    # Read the band data
                    band_data = src.read(band_idx)

                    # Do nearest neighbor interpolation for lake values
                    non_nan_indices = np.where(~np.isnan(band_data))
                    points = np.column_stack((non_nan_indices[1], non_nan_indices[0]))
                    values = band_data[non_nan_indices]
                    rows, cols = band_data.shape
                    grid_x, grid_y = np.meshgrid(np.arange(cols), np.arange(rows))
                    interpolated_values = griddata(points, values, (grid_x, grid_y), method='nearest')
                    band_data[np.isnan(band_data)] = interpolated_values[np.isnan(band_data)]

                    year, month, day = layer_name.split("-")
                    # Create a new raster file for each band with the corresponding layer name
                    output_raster_path = f"{output_directory}/{out_var}{int(year)}{int(month):02d}{int(day):02d}.tif"

                    # Write the band data to the new raster file with the same metadata
                    with rasterio.open(output_raster_path, 'w', **src.profile) as dst:
                        dst.write(band_data, 1)
        

In [2]:
import datetime

def generate_days(year):
    start_date = datetime.date(year, 1, 1)
    end_date = datetime.date(year, 12, 31)
    delta = datetime.timedelta(days=1)

    days_in_year = []
    current_date = start_date
    while current_date <= end_date:
        days_in_year.append(current_date.strftime("%Y-%m-%d"))
        current_date += delta

    return days_in_year

In [4]:
generate_days(1950)[0].split("-")

['1950', '01', '01']